In [1]:
import warnings
warnings.filterwarnings("ignore")

import importlib
from pathlib import Path
from argparse import ArgumentParser
from glob import glob 

import torch
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import ModelCheckpoint

import lightning as L
from lightning import LightningModule

from datasets import Dataset, load_dataset

import torchmetrics
from sklearn.model_selection import train_test_split

import wandb

In [2]:
import numpy as np
import pandas as pd

In [3]:
from pathlib import Path
from glob import glob

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [5]:
# dataset = './data/wesad_model_20s'
# dataset = './Stress_Detection_ECG/content/data/Dataset'

In [6]:
# files = glob(f'{dataset}/*.csv')

In [7]:
# train_participants = [Path(file).stem for file in files if Path(file).stem != 'S17']
# test_participants = [Path(file).stem for file in files if Path(file).stem == 'S17']

In [8]:
import datasets

In [9]:
from sia.datasets.edouard_dataset import Dataset
from sia.models.fully_connected_neural_network import Model

In [10]:
from pytorch_lightning.trainer.states import TrainerFn

In [11]:
class Base(Model):
    def __init__(self, batch_size: int, learning_rate: float, num_workers: int):
        self.save_hyperparameters()
        super().__init__()

    def prepare_data(self):
        self.train_data = datasets.load_from_disk('./Stress_Detection_ECG/wesad/train')
        self.train_data = self.train_data.with_format("torch")
        self.test_data = datasets.load_from_disk('./Stress_Detection_ECG/wesad/test')
        self.test_data = self.test_data.with_format("torch")

    def setup(self, stage):
        if stage == TrainerFn.FITTING:
            self.dataset = Dataset(self.train_data)
        elif stage == TrainerFn.TESTING:
            self.dataset = Dataset(self.test_data)

        self.train_accuracy = torchmetrics.classification.BinaryAccuracy()
        self.train_f1score = torchmetrics.classification.BinaryF1Score()
        self.train_precision = torchmetrics.classification.BinaryPrecision()
        self.train_loss = torch.nn.BCELoss()

        self.test_accuracy = torchmetrics.classification.BinaryAccuracy()
        self.test_f1score = torchmetrics.classification.BinaryF1Score()
        self.test_precision = torchmetrics.classification.BinaryPrecision()
        self.test_loss = torch.nn.BCELoss()

    def forward(self, x):
        embedding = self.layers(x)
        return embedding.squeeze()
    
    def training_step(self, batch, batch_idx):
        y, y_hat = self._step(batch, batch_idx)

        step_loss = self.train_loss(y_hat, y)
        self.train_accuracy.update(y_hat.squeeze(), y)
        self.train_f1score.update(y_hat.squeeze(), y)
        self.train_precision.update(y_hat.squeeze(), y)

        if wandb.run is not None:
            wandb.log({"accuracy": self.train_accuracy.compute(), "precision": self.train_precision.compute(), "loss": step_loss, "f1": self.train_f1score.compute()})
        
        return step_loss
    
    def test_step(self, batch, batch_idx):
        y, y_hat = self._step(batch, batch_idx)

        step_loss = self.test_loss(y_hat, y)
        self.test_accuracy.update(y_hat.squeeze(), y)
        self.test_f1score.update(y_hat.squeeze(), y)
        self.test_precision.update(y_hat.squeeze(), y)

        if wandb.run is not None:
            wandb.log({"test_accuracy": self.test_accuracy.compute(), "test_precision": self.test_precision.compute(), "test_loss": step_loss, "test_f1": self.test_f1score.compute()})
        
        return step_loss

    def _step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return y, y_hat
    
    def configure_optimizers(self) -> L.pytorch.utilities.types.OptimizerLRScheduler:
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate, betas=(0.9, 0.999))
    
    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.hparams.batch_size, num_workers=self.hparams.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.hparams.batch_size, num_workers=self.hparams.num_workers, pin_memory=True)

In [12]:
wandb.init(
    project="wesad-stress-detection",
    config={
        "epochs": 15,
        "architecture": "Fully Connected Neural Network",
        "dataset": "WESAD"
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alex-antonides. Use `wandb login --relogin` to force relogin


In [13]:
trainer = L.Trainer(
    max_epochs=15, 
    accelerator="auto", 
    devices="auto", 
    strategy="auto", 
    profiler="simple",
    default_root_dir=f"./checkpoints/edouard"
)

tuner = L.pytorch.tuner.Tuner(
    trainer
)

trainer.fit(
    model=Base(
        batch_size=32, 
        learning_rate=0.0001, 
        num_workers=4
    ),
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
----------------------------------------------------
0 | layers          | Sequential      | 11.5 K
1 | train_accuracy  | BinaryAccuracy  | 0     
2 | train_f1score   | BinaryF1Score   | 0     
3 | train_precision | BinaryPrecision | 0     
4 | train_loss      | BCELoss         | 0     
5 | test_accuracy   | BinaryAccuracy  | 0     
6 | test_f1score    | BinaryF1Score   | 0     
7 | test_precision  | BinaryPrecision | 0     
8 | test_loss       | BCELoss         | 0     
----------------------------------------------------
11.5 K    Trainable params
0         Non-trainable params
11.5 K    Total params
0.046     Total estimated model params size (MB)


Epoch 1:  81%|████████▏ | 456/560 [00:10<00:02, 44.26it/s, v_num=46]

In [14]:
if wandb.run is not None:
    wandb.finish()

accuracy,█▂▃▁▂▂▂▂▂▁▁▁▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
f1,█▄▃▁▃▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▅▄▃▂▅▆▄▆▆▆▅▄▆█▆▄▅▃▂▅▄▂▄▄▆▅▃▃▅▃▄▄▄▃▃▃▅▅▁▄
precision,█▄▃▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.4968
f1,0.50433
loss,0.77643
precision,0.50424
